In [1]:
%load_ext autoreload

In [4]:
import xlrd
import pandas as pd

from datetime import datetime as dt
from datetime import timedelta
from dateutil.relativedelta import relativedelta

from model import plan_var as V
from model import plan_loader as P
from model import reformatter as R

from plotly.subplots import make_subplots
import plotly.graph_objects as go

from gantt_bar import *

import holidays

In [3]:
d = dt.today()
d.weekday()

4

In [11]:
d = dt(year=2020, month=1, day=1)
ru_hollidays = holidays.RU()
for i in range(370):
    if d in ru_hollidays:
        print(d)
    d = d + relativedelta(days=1)
    

2020-01-01 00:00:00
2020-01-02 00:00:00
2020-01-03 00:00:00
2020-01-04 00:00:00
2020-01-05 00:00:00
2020-01-06 00:00:00
2020-01-07 00:00:00
2020-01-08 00:00:00
2020-02-23 00:00:00
2020-03-08 00:00:00
2020-05-01 00:00:00
2020-05-09 00:00:00
2020-06-12 00:00:00
2020-11-04 00:00:00
2021-01-01 00:00:00
2021-01-02 00:00:00
2021-01-03 00:00:00
2021-01-04 00:00:00


In [21]:
dct = {1:1, 2:2}
d = dt.today() - relativedelta(days=14)
d = d.replace(hour=0,minute=0,second=0, microsecond=0)
dct['x0'] = d + relativedelta(days=1)
dct

{1: 1, 2: 2, 'x0': datetime.datetime(2020, 9, 12, 0, 0)}

In [3]:
%autoreload
plan_loader = P.PlanLoader('план.xlsx')
plan_loader.delete_rows_ready()

In [4]:
plan_loader.df.head(2)

,Заливка,Заливка_1,Навеска,Навеска_1,Передача ЭМУ,Сборка щитов,Монтаж СН,Шина УВР/ТР-Р,"Сборка 0,4 кВ","Втор-ка 0,4 кВ",Сборка 6/10 кВ,Втор-ка 6/10 кВ,"Уст-ка 0,4/10кВ",Установ-ка RM-6,Наладка,Комплек-тация,ОТГРУЗКА,ОТГРУЗКА_1
1250,готов,готов,20.11,20-25.11,готов,20-25.11,20-23.11,готов,готов,27-29.11,нет,нет,19.11,20.11,05.12,06.12,2019-12-06,нет
1253,готов,готов,26.11,26-27.11,25.11,26-28.11,25-26.11,готов,готов,02-04.13,нет,нет,25.12,27.12,05.13,06.13,2019-12-10,нет


In [5]:
%autoreload
ref = R.PlanReformatter(plan_loader.df)
gant = ref.get_df_for_gantt()
gant

В зз 1253 столбец Втор-ка 0,4 кВ имеет неверное значение
В зз 1253 столбец Наладка имеет неверное значение
В зз 1253 столбец Комплек-тация имеет неверное значение
В зз 1256 столбец Монтаж СН имеет неверное значение


,Task,Start,Finish,zakaz
0,Заливка_НМ,2020-09-18 08:20:00,2020-09-23 16:20:00,1333
1,Заливка_ВМ,2020-09-24 08:20:00,2020-10-01 16:20:00,1333
2,Навеска_НМ,2020-10-15 08:20:00,2020-10-15 16:20:00,1333
3,Навеска_ВМ,2020-10-15 08:20:00,2020-10-20 16:20:00,1333
4,ЭМУ,2020-10-19 08:20:00,2020-10-19 16:20:00,1333
...,...,...,...,...
139,УсткаНН,2020-11-23 08:20:00,2020-11-23 16:20:00,1340
140,УсткаВВ,2020-11-23 08:20:00,2020-11-23 16:20:00,1340
141,УсткаRM,2020-11-23 08:20:00,2020-11-23 16:20:00,1340
142,Наладка,2020-11-25 08:20:00,2020-11-25 16:20:00,1340


In [13]:
gant

,Task,Start,Finish,zakaz
0,Заливка_НМ,2020-09-18 08:20:00,2020-09-23 16:20:00,1333
1,Заливка_ВМ,2020-09-24 08:20:00,2020-10-01 16:20:00,1333
2,Навеска_НМ,2020-10-15 08:20:00,2020-10-15 16:20:00,1333
3,Навеска_ВМ,2020-10-15 08:20:00,2020-10-20 16:20:00,1333
4,ЭМУ,2020-10-19 08:20:00,2020-10-19 16:20:00,1333
...,...,...,...,...
139,УсткаНН,2020-11-23 08:20:00,2020-11-23 16:20:00,1340
140,УсткаВВ,2020-11-23 08:20:00,2020-11-23 16:20:00,1340
141,УсткаRM,2020-11-23 08:20:00,2020-11-23 16:20:00,1340
142,Наладка,2020-11-25 08:20:00,2020-11-25 16:20:00,1340


In [16]:
for _, i in gant.head(5).iterrows():
    print(i)

Task               Заливка_НМ
Start     2020-09-18 08:20:00
Finish    2020-09-23 16:20:00
zakaz                    1333
Name: 0, dtype: object
Task               Заливка_ВМ
Start     2020-09-24 08:20:00
Finish    2020-10-01 16:20:00
zakaz                    1333
Name: 1, dtype: object
Task               Навеска_НМ
Start     2020-10-15 08:20:00
Finish    2020-10-15 16:20:00
zakaz                    1333
Name: 2, dtype: object
Task               Навеска_ВМ
Start     2020-10-15 08:20:00
Finish    2020-10-20 16:20:00
zakaz                    1333
Name: 3, dtype: object
Task                      ЭМУ
Start     2020-10-19 08:20:00
Finish    2020-10-19 16:20:00
zakaz                    1333
Name: 4, dtype: object


In [28]:
l1 = ['Комплект', 'Наладка', 'УсткаRM', 'УсткаВВ', 'УсткаНН', 'ВторВВ', 'СборкаВВ', 'ВторНН', 'СборкаНН', 'Шина_ТР', 'ШинаУВР', 'Монтаж_СН', 'СборкаЩитов', 'ЭМУ', 'Навеска_ВМ', 'Навеска_НМ', 'Заливка_ВМ', 'Заливка_НМ']
l2 = ['Наладка', 'УсткаRM', 'УсткаВВ', 'УсткаНН', 'ВторВВ', 'СборкаВВ', 'ВторНН', 'СборкаНН', 'Шина_ТР', 'ШинаУВР', 'Монтаж_СН', 'СборкаЩитов', 'ЭМУ', 'Навеска_ВМ', 'Навеска_НМ', 'Заливка_ВМ', 'Заливка_НМ']
i=0
while i < len(l1):
            if l1[i] not in l2:
                print(i, l1[i])
                l1.pop(i)
            i+=1

0 Комплект


In [21]:
full_y_list = ref.get_jobs_names()[::-1]
full_y_list

['Комплект',
 'Наладка',
 'УсткаRM',
 'УсткаВВ',
 'УсткаНН',
 'ВторВВ',
 'СборкаВВ',
 'ВторНН',
 'СборкаНН',
 'Шина_ТР',
 'ШинаУВР',
 'Монтаж_СН',
 'СборкаЩитов',
 'ЭМУ',
 'Навеска_ВМ',
 'Навеска_НМ',
 'Заливка_ВМ',
 'Заливка_НМ']

In [29]:
gant[gant['Task'].isin(full_y_list[0:2])]

,Task,Start,Finish,zakaz
14,Наладка,2020-10-28 08:20:00,2020-10-28 16:20:00,1333
15,Комплект,2020-10-30 08:20:00,2020-10-30 16:20:00,1333
27,Наладка,2020-10-25 08:20:00,2020-10-29 16:20:00,1326
28,Комплект,2020-10-29 08:20:00,2020-10-29 16:20:00,1326
31,Наладка,2020-10-29 08:20:00,2020-10-29 16:20:00,1341
32,Комплект,2020-10-29 08:20:00,2020-10-29 16:20:00,1341
47,Наладка,2020-11-05 08:20:00,2020-11-05 16:20:00,1334
48,Комплект,2020-11-05 08:20:00,2020-11-05 16:20:00,1334
63,Наладка,2020-11-06 08:20:00,2020-11-06 16:20:00,1337
64,Комплект,2020-11-09 08:20:00,2020-11-09 16:20:00,1337


In [11]:
%autoreload
dd = data = ref.get_df_for_gantt(2,1100)
dd

NameError: name 'ref' is not defined

In [12]:
%autoreload
fig = go.Figure()
b = GanttBar()

data = ref.get_df_for_gantt(17,10)
for i in range(data.shape[0]):
    fig.add_trace(b.get_bar(data['Start'][i], data['Finish'][i],
                    l.index(data['Task'][i]), group=int(data['zakaz'][i])))
    
fig.update_layout(
    yaxis = dict(
        tickmode = 'array',
        tickvals = [n for n in range(len(l))],
        ticktext = l
    )
)

fig.update_xaxes(
    dtick='D1',
    range=[dt.today() - timedelta(days=7),
          dt.today() + timedelta(days=30),]
    )
fig.show()

NameError: name 'go' is not defined

In [13]:
dt.today() - timedelta(days=10)

datetime.datetime(2020, 9, 11, 12, 4, 47, 820685)

In [14]:
data[data['Task'] == 'УсткаRM']

NameError: name 'data' is not defined